# 分散表現による文書類似度の推定
参考：https://qiita.com/sonoisa/items/775ac4c7871ced6ed4c3

In [5]:
import os
from typing import List

import MeCab
import fasttext
import fasttext.util
import numpy as np
from IPython.display import Image
from sklearn.metrics.pairwise import cosine_similarity

## Configs

In [6]:
FASTTEXT_VEC = 'cc.ja.300.bin'
m = MeCab.Tagger("-Owakati")  # TODO:わかち書きしているので、品詞指定で特徴語を抽出するようにする必要がありそう
ILLUST_YA_CSV_PATH = '/Users/rikeda/Downloads/hackday/illust.csv'

### いらすとやの見出しの格納

In [7]:
# 試しにぐっちが集めたいらすとやの見出しを使ってみる
import csv
with open(ILLUST_YA_CSV_PATH, 'r') as f:
    reader = csv.reader(f)
    illusts = [(m.parse(row[1]), row[2]) for row in reader]
print(illusts[:3])

[('お金 の 単位 の キャラクター （ 強い 円 ） \n', 'https://4.bp.blogspot.com/-jN5C5Y4xN9s/V2zF20D4INI/AAAAAAAA8AQ/sVYu-ezC_3UKtX4n75AaHUqEskfQQnS7QCLcB/s150/money_character_strong_yen.png'), ('風邪 薬 の イラスト \n', 'https://2.bp.blogspot.com/-57P4j4ba84E/VyNdckfKDVI/AAAAAAAA6No/00EjX61RCQwiFfvQjglB7M55X2xsO65MQCLcB/s400/medicine_kaze.png'), ('青い バラ の イラスト \n', 'https://3.bp.blogspot.com/-D1ojODeU7lg/U0pS6kDirII/AAAAAAAAe_Y/LU00iN_kGs0/s400/rose_blue.png')]


### fasttextモデルのロード

In [8]:
fasttext.util.download_model('ja', if_exists='ignore')  # 学習済みモデルがなければdownload
ft = fasttext.load_model(FASTTEXT_VEC)
ft.get_dimension()

300

In [9]:
ft.get_nearest_neighbors('Google', k=3)

[(0.755213737487793, 'Goole'),
 (0.7355369329452515, 'Goolge'),
 (0.7275019288063049, 'グーグル')]

In [10]:
# SWEM(Simple Word-Embedding based Models)
# https://arxiv.org/abs/1805.09843v1

def _get_word_vectors(words: List[str]):
    return np.array([ft.get_word_vector(word) for word in words])

def average_pooling(words: List[str]):
    return np.mean(_get_word_vectors(words), axis=0)

def max_pooling(words: List[str]):
    return np.max(_get_word_vectors(words), axis=0)

In [22]:
sentence = m.parse("東京都内で２５日、新型コロナウイルスの感染者が新たに４８人確認されたことがわかった。１日当たりの感染者は２４日（５５人）を下回ったが、直近１週間の平均は約３９人と高水準が続いている。").replace(' \n', '')
a = np.array([max_pooling(sentence.split(' ')[:-1])])
b = np.array([max_pooling(illust[0].split(' ')) for illust in illusts])  # いらすとやの見出しのベクトル集合
similarity = cosine_similarity(a, b)[0]

In [23]:
top3_indices = np.argsort(similarity)[::-1][:3]

print(similarity[top3_indices.tolist()[0]])
print(illusts[top3_indices.tolist()[0]][0])
Image(url =  illusts[top3_indices.tolist()[0]][1])

0.8963355
地域 を またい だ 感染 拡大 の イラスト （ 感染 地域 から ） 



In [26]:
print(similarity[top3_indices.tolist()[1]])
print(illusts[top3_indices.tolist()[1]][0])
Image(url =  illusts[top3_indices.tolist()[1]][1])

0.8937279
スマート フォン で チャート を 見る 人 の イラスト （ 上昇 ） 



In [25]:
print(similarity[top3_indices.tolist()[2]])
print(illusts[top3_indices.tolist()[2]][0])
Image(url =  illusts[top3_indices.tolist()[2]][1])

0.89311564
「 感染 症 拡大 防止 に ご 協力 ください 」 の イラスト （ 会社 員 ） 



In [27]:
sentence = "東京都内で２５日、新型コロナウイルスの感染者が新たに４８人確認されたことがわかった。"
a = np.array([average_pooling(m.parse(sentence).split(' ')[:-1])])
b = np.array([average_pooling(illust[0].split(' ')) for illust in illusts])  # いらすとやの見出しのベクトル集合
similarity = cosine_similarity(a, b)[0]
top3_indices = np.argsort(similarity)[::-1][:3]

print(similarity[top3_indices.tolist()[0]])
print(illusts[top3_indices.tolist()[0]][0])
Image(url =  illusts[top3_indices.tolist()[0]][1])

0.86407477
時間 を 忘れ て しまっ た 人 の イラスト （ 男性 ） 

